# Data Wrangling Exercises

### 1. Acquire data from mySQL using the python module to connect and query. You will want to end with a single dataframe. Make sure to include: the logerror, all fields related to the properties that are available. You will end up using all the tables in the database.

- Be sure to do the correct join (inner, outer, etc.). We do not want to eliminate properties purely because they may have a null value for airconditioningtypeid.

- Only include properties with a transaction in 2017, and include only the last transaction for each property (so no duplicate property ID's), along with zestimate error and date of transaction.

- Only include properties that include a latitude and longitude value.

In [1]:
import pandas as pd
import numpy as np
import os
from env import host, user, password

def get_connection(db, user=user, host=host, password=password):
    '''
    This function uses my info from my env file to
    create a connection url to access the Codeup db.
    It takes in a string name of a database as an argument.
    '''
    return f'mysql+pymysql://{user}:{password}@{host}/{db}'
    
    
def new_zillow_data():
    '''
    This function reads all tables and columns into a dataframe, including only properties with lat/long
    data and had transactions in 2017. Properties with multiple transactions 
    will display only the most recent transaction.
    '''
    sql_query = """
                SELECT p.*, m.logerror, m.transactiondate, ac.airconditioningdesc, arch.architecturalstyledesc, b.buildingclassdesc, heat.heatingorsystemdesc, pt.propertylandusedesc, s.storydesc, c.typeconstructiondesc
                FROM properties_2017 as p
                JOIN predictions_2017 as m USING(parcelid)
                LEFT JOIN airconditioningtype as ac USING(airconditioningtypeid)
                LEFT JOIN architecturalstyletype as arch USING(architecturalstyletypeid)
                LEFT JOIN buildingclasstype as b USING(buildingclasstypeid)
                LEFT JOIN heatingorsystemtype as heat USING(heatingorsystemtypeid)
                LEFT JOIN propertylandusetype as pt USING(propertylandusetypeid)
                LEFT JOIN storytype as s USING(storytypeid)
                LEFT JOIN typeconstructiontype as c USING(typeconstructiontypeid)
                LEFT JOIN unique_properties as u USING(parcelid)
                INNER JOIN (SELECT p.parcelid, MAX(transactiondate) AS maxdate FROM properties_2017 as p JOIN predictions_2017 USING(parcelid) GROUP BY p.parcelid, logerror) md ON p.parcelid = md.parcelid AND transactiondate = maxdate
                WHERE transactiondate LIKE '2017%%' AND latitude IS NOT NULL AND longitude IS NOT NULL
                """
    
    # Read in DataFrame from Codeup db.
    df = pd.read_sql(sql_query, get_connection('zillow'))
    
    return df

def get_zillow_data():
    '''
    This function reads in zillow data from the Codeup database, writes data to
    a csv file if a local file does not exist, and returns a df.
    '''
    if os.path.isfile('zillow.csv'):
        
        # If csv file exists read in data from csv file.
        df = pd.read_csv('zillow.csv', index_col=0)
        
    else:
        
        # Read fresh data from db into a DataFrame
        df = new_zillow_data()
        
        # Cache data
        df.to_csv('zillow.csv')
        
    return df


### 2. Summarize your data (summary stats, info, dtypes, shape, distributions, value_counts, etc.)

In [2]:
df = get_zillow_data()

/opt/homebrew/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3357: DtypeWarning: Columns (64) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


In [3]:
cols = []
count = 1
for column in df.columns:
    if column == 'id':
        cols.append(f'id_{count}')
        count+=1
        continue
    cols.append(column)
df.columns = cols

In [4]:
for col in df.columns:
    print(df[col].value_counts())
    print('\n---------------------------')

1968735    3
709378     3
694050     2
82724      2
2093710    2
          ..
1344133    1
565889     1
1878656    1
479871     1
657407     1
Name: id_1, Length: 77381, dtype: int64

---------------------------
12612211    3
11991059    3
17295416    2
12118682    2
12870253    2
           ..
14738121    1
10808008    1
17084671    1
12485317    1
12853244    1
Name: parcelid, Length: 77381, dtype: int64

---------------------------
1.0     23185
13.0     1575
5.0       167
11.0       53
9.0        26
Name: airconditioningtypeid, dtype: int64

---------------------------
7.0     172
8.0      19
21.0      7
2.0       5
3.0       3
Name: architecturalstyletypeid, dtype: int64

---------------------------
640.0     2
900.0     2
515.0     2
700.0     2
273.0     2
912.0     2
100.0     2
3560.0    1
786.0     1
204.0     1
1416.0    1
512.0     1
38.0      1
1809.0    1
168.0     1
280.0     1
1252.0    1
604.0     1
1218.0    1
396.0     1
645.0     1
314.0     1
819.0     1
600.0     

In [5]:
df.shape

(77574, 68)

In [6]:
df.dtypes

id_1                          int64
parcelid                      int64
airconditioningtypeid       float64
architecturalstyletypeid    float64
basementsqft                float64
                             ...   
buildingclassdesc            object
heatingorsystemdesc          object
propertylandusedesc          object
storydesc                    object
typeconstructiondesc         object
Length: 68, dtype: object

In [7]:
df.describe()

,id_1,parcelid,airconditioningtypeid,architecturalstyletypeid,basementsqft,bathroomcnt,bedroomcnt,buildingclasstypeid,buildingqualitytypeid,calculatedbathnbr,...,numberofstories,fireplaceflag,structuretaxvaluedollarcnt,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,taxdelinquencyyear,censustractandblock,logerror
count,7.757400e+04,7.757400e+04,25006.000000,206.000000,50.000000,77574.000000,77574.000000,15.000000,49808.000000,76959.000000,...,17596.000000,172.0,7.745900e+04,7.757300e+04,77574.0,7.757200e+04,77569.000000,2900.000000,7.732700e+04,77574.000000
mean,1.495340e+06,1.300630e+07,1.812045,7.388350,679.720000,2.298521,3.053252,3.933333,6.533830,2.316389,...,1.434246,1.0,1.892829e+05,4.901477e+05,2016.0,3.011468e+05,5995.855691,14.088276,6.049666e+13,0.016805
std,8.609689e+05,3.478043e+06,2.965823,2.734542,689.703546,0.996706,1.140447,0.258199,1.722041,0.979690,...,0.544518,0.0,2.304149e+05,6.538059e+05,0.0,4.927317e+05,7628.912331,2.181281,1.533386e+12,0.170742
min,3.490000e+02,1.071186e+07,1.000000,2.000000,38.000000,0.000000,0.000000,3.000000,1.000000,1.000000,...,1.000000,1.0,4.400000e+01,1.000000e+03,2016.0,1.610000e+02,19.920000,3.000000,6.037101e+13,-4.655420
25%,7.520875e+05,1.153819e+07,1.000000,7.000000,273.000000,2.000000,2.000000,4.000000,6.000000,2.000000,...,1.000000,1.0,8.418200e+04,2.068980e+05,2016.0,8.529200e+04,2712.610000,14.000000,6.037311e+13,-0.024310
50%,1.498169e+06,1.253050e+07,1.000000,7.000000,515.000000,2.000000,3.000000,4.000000,6.000000,2.000000,...,1.000000,1.0,1.364070e+05,3.588790e+05,2016.0,2.031810e+05,4448.300000,15.000000,6.037603e+13,0.006673
75%,2.240588e+06,1.421124e+07,1.000000,7.000000,796.500000,3.000000,4.000000,4.000000,8.000000,3.000000,...,2.000000,1.0,2.187340e+05,5.690000e+05,2016.0,3.667610e+05,6926.820000,15.000000,6.059042e+13,0.039292
max,2.982274e+06,1.676893e+08,13.000000,21.000000,3560.000000,18.000000,16.000000,4.000000,12.000000,18.000000,...,6.000000,1.0,1.142179e+07,4.906124e+07,2016.0,4.895220e+07,586639.300000,99.000000,4.830301e+14,5.262999


### 3. Write a function that takes in a dataframe of observations and attributes and returns a dataframe where each row is an atttribute name, the first column is the number of rows with missing values for that attribute, and the second column is percent of total rows that have missing values for that attribute. Run the function and document takeaways from this on how you want to handle missing values.

In [8]:
def missing_values(df):
    df_nulls = pd.DataFrame({'num_rows_missing':df.isnull().sum(),'pct_rows_missing':df.isnull().sum()/len(df)*100})
    return df_nulls

In [9]:
df_nulls = missing_values(df)

In [10]:
df_nulls[df_nulls.pct_rows_missing > 70]

,num_rows_missing,pct_rows_missing
architecturalstyletypeid,77368,99.734447
basementsqft,77524,99.935545
buildingclasstypeid,77559,99.980664
decktypeid,76960,99.208498
finishedfloor1squarefeet,71539,92.220332
finishedsquarefeet13,77532,99.945858
finishedsquarefeet15,74547,96.097919
finishedsquarefeet50,71539,92.220332
finishedsquarefeet6,77188,99.502411
fireplacecnt,69287,89.317297


Takeaways: drop high percentage columns and impute low percentage columns

### 4. Write a function that takes in a dataframe and returns a dataframe with 3 columns: the number of columns missing, percent of columns missing, and number of rows with n columns missing. Run the function and document takeaways from this on how you want to handle missing values.

In [11]:
def missing_cols(df):
    df_missing_cols = pd.DataFrame(df.isnull().sum(axis =1), columns = ['num_cols_missing']).reset_index().groupby('num_cols_missing').count().reset_index().rename(columns = {'index': 'num_rows'})
    df_missing_cols['pct_cols_missing'] = df_missing_cols.num_cols_missing/df.shape[1]
    return df_missing_cols

In [12]:
df_missing_cols = missing_cols(df)
df_missing_cols.head()

,num_cols_missing,num_rows,pct_cols_missing
0,23,2,0.338235
1,24,13,0.352941
2,25,24,0.367647
3,26,65,0.382353
4,27,316,0.397059


## Prepare



### 1. Remove any properties that are likely to be something other than single unit properties. (e.g. no duplexes, no land/lot, ...). There are multiple ways to estimate that a property is a single unit, and there is not a single "right" answer. But for this exercise, do not purely filter by unitcnt as we did previously. Add some new logic that will reduce the number of properties that are falsely removed. You might want to use # bedrooms, square feet, unit type or the like to then identify those with unitcnt not defined.

In [13]:
df = df[df.propertylandusetypeid == 261]

In [14]:
df.head()

,id_1,parcelid,airconditioningtypeid,architecturalstyletypeid,basementsqft,bathroomcnt,bedroomcnt,buildingclasstypeid,buildingqualitytypeid,calculatedbathnbr,...,censustractandblock,logerror,transactiondate,airconditioningdesc,architecturalstyledesc,buildingclassdesc,heatingorsystemdesc,propertylandusedesc,storydesc,typeconstructiondesc
0,1727539,14297519,NaN,NaN,NaN,3.5,4.0,NaN,NaN,3.5,...,6.059063e+13,0.025595,2017-01-01,NaN,NaN,NaN,NaN,Single Family Residential,NaN,NaN
1,1387261,17052889,NaN,NaN,NaN,1.0,2.0,NaN,NaN,1.0,...,6.111001e+13,0.055619,2017-01-01,NaN,NaN,NaN,NaN,Single Family Residential,NaN,NaN
2,11677,14186244,NaN,NaN,NaN,2.0,3.0,NaN,NaN,2.0,...,6.059022e+13,0.005383,2017-01-01,NaN,NaN,NaN,NaN,Single Family Residential,NaN,NaN
3,2288172,12177905,NaN,NaN,NaN,3.0,4.0,NaN,8.0,3.0,...,6.037300e+13,-0.103410,2017-01-01,NaN,NaN,NaN,Central,Single Family Residential,NaN,NaN
6,781532,12095076,1.0,NaN,NaN,3.0,4.0,NaN,9.0,3.0,...,6.037461e+13,-0.001011,2017-01-01,Central,NaN,NaN,Central,Single Family Residential,NaN,NaN


### 2. Create a function that will drop rows or columns based on the percent of values that are missing: handle_missing_values(df, prop_required_column, prop_required_row).

In [15]:
def handle_missing_values(df, prop_required_column = .5, prop_required_row = .70):
    threshold = int(round(prop_required_column*len(df.index),0))
    df.dropna(axis=1, thresh=threshold, inplace=True)
    threshold = int(round(prop_required_row*len(df.columns),0))
    df.dropna(axis=0, thresh=threshold, inplace=True)
    return df

In [16]:
df = handle_missing_values(df)

In [17]:
df.head()

,id_1,parcelid,bathroomcnt,bedroomcnt,buildingqualitytypeid,calculatedbathnbr,calculatedfinishedsquarefeet,finishedsquarefeet12,fips,fullbathcnt,...,structuretaxvaluedollarcnt,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,censustractandblock,logerror,transactiondate,heatingorsystemdesc,propertylandusedesc
0,1727539,14297519,3.5,4.0,NaN,3.5,3100.0,3100.0,6059.0,3.0,...,485713.0,1023282.0,2016.0,537569.0,11013.72,6.059063e+13,0.025595,2017-01-01,NaN,Single Family Residential
1,1387261,17052889,1.0,2.0,NaN,1.0,1465.0,1465.0,6111.0,1.0,...,88000.0,464000.0,2016.0,376000.0,5672.48,6.111001e+13,0.055619,2017-01-01,NaN,Single Family Residential
2,11677,14186244,2.0,3.0,NaN,2.0,1243.0,1243.0,6059.0,2.0,...,85289.0,564778.0,2016.0,479489.0,6488.30,6.059022e+13,0.005383,2017-01-01,NaN,Single Family Residential
3,2288172,12177905,3.0,4.0,8.0,3.0,2376.0,2376.0,6037.0,3.0,...,108918.0,145143.0,2016.0,36225.0,1777.51,6.037300e+13,-0.103410,2017-01-01,Central,Single Family Residential
6,781532,12095076,3.0,4.0,9.0,3.0,2962.0,2962.0,6037.0,3.0,...,276684.0,773303.0,2016.0,496619.0,9516.26,6.037461e+13,-0.001011,2017-01-01,Central,Single Family Residential


In [18]:
cols_to_remove = ['id','calculatedbathnbr', 'finishedsquarefeet12', 'fullbathcnt', 'heatingorsystemtypeid','propertycountylandusecode', 'propertylandusetypeid','propertyzoningdesc',  'censustractandblock', 'propertylandusedesc', 'unitcnt']

In [19]:
def drop_cols(df, cols_to_drop):
    df.drop(columns = cols_to_drop, inplace = True)
    return df

In [20]:
df = drop_cols(df,cols_to_remove)

In [21]:
df.head()

,parcelid,bathroomcnt,bedroomcnt,buildingqualitytypeid,calculatedfinishedsquarefeet,fips,latitude,longitude,lotsizesquarefeet,rawcensustractandblock,...,roomcnt,yearbuilt,structuretaxvaluedollarcnt,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,logerror,transactiondate,heatingorsystemdesc
0,14297519,3.5,4.0,NaN,3100.0,6059.0,33634931.0,-117869207.0,4506.0,6.059063e+07,...,0.0,1998.0,485713.0,1023282.0,2016.0,537569.0,11013.72,0.025595,2017-01-01,NaN
1,17052889,1.0,2.0,NaN,1465.0,6111.0,34449266.0,-119281531.0,12647.0,6.111001e+07,...,5.0,1967.0,88000.0,464000.0,2016.0,376000.0,5672.48,0.055619,2017-01-01,NaN
2,14186244,2.0,3.0,NaN,1243.0,6059.0,33886168.0,-117823170.0,8432.0,6.059022e+07,...,6.0,1962.0,85289.0,564778.0,2016.0,479489.0,6488.30,0.005383,2017-01-01,NaN
3,12177905,3.0,4.0,8.0,2376.0,6037.0,34245180.0,-118240722.0,13038.0,6.037300e+07,...,0.0,1970.0,108918.0,145143.0,2016.0,36225.0,1777.51,-0.103410,2017-01-01,Central
6,12095076,3.0,4.0,9.0,2962.0,6037.0,34145202.0,-118179824.0,63000.0,6.037461e+07,...,0.0,1950.0,276684.0,773303.0,2016.0,496619.0,9516.26,-0.001011,2017-01-01,Central


In [22]:
df.drop(columns = 'heatingorsystemdesc', inplace = True)

In [23]:
df.drop(columns = 'buildingqualitytypeid', inplace = True)

In [24]:
df.head()

,parcelid,bathroomcnt,bedroomcnt,calculatedfinishedsquarefeet,fips,latitude,longitude,lotsizesquarefeet,rawcensustractandblock,regionidcity,...,regionidzip,roomcnt,yearbuilt,structuretaxvaluedollarcnt,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,logerror,transactiondate
0,14297519,3.5,4.0,3100.0,6059.0,33634931.0,-117869207.0,4506.0,6.059063e+07,53571.0,...,96978.0,0.0,1998.0,485713.0,1023282.0,2016.0,537569.0,11013.72,0.025595,2017-01-01
1,17052889,1.0,2.0,1465.0,6111.0,34449266.0,-119281531.0,12647.0,6.111001e+07,13091.0,...,97099.0,5.0,1967.0,88000.0,464000.0,2016.0,376000.0,5672.48,0.055619,2017-01-01
2,14186244,2.0,3.0,1243.0,6059.0,33886168.0,-117823170.0,8432.0,6.059022e+07,21412.0,...,97078.0,6.0,1962.0,85289.0,564778.0,2016.0,479489.0,6488.30,0.005383,2017-01-01
3,12177905,3.0,4.0,2376.0,6037.0,34245180.0,-118240722.0,13038.0,6.037300e+07,396551.0,...,96330.0,0.0,1970.0,108918.0,145143.0,2016.0,36225.0,1777.51,-0.103410,2017-01-01
6,12095076,3.0,4.0,2962.0,6037.0,34145202.0,-118179824.0,63000.0,6.037461e+07,47019.0,...,96293.0,0.0,1950.0,276684.0,773303.0,2016.0,496619.0,9516.26,-0.001011,2017-01-01


In [25]:
df = df[df.taxvaluedollarcnt < 5_000_000]

In [26]:
df[df.calculatedfinishedsquarefeet < 8000]

,parcelid,bathroomcnt,bedroomcnt,calculatedfinishedsquarefeet,fips,latitude,longitude,lotsizesquarefeet,rawcensustractandblock,regionidcity,...,regionidzip,roomcnt,yearbuilt,structuretaxvaluedollarcnt,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,logerror,transactiondate
0,14297519,3.5,4.0,3100.0,6059.0,33634931.0,-117869207.0,4506.0,6.059063e+07,53571.0,...,96978.0,0.0,1998.0,485713.0,1023282.0,2016.0,537569.0,11013.72,0.025595,2017-01-01
1,17052889,1.0,2.0,1465.0,6111.0,34449266.0,-119281531.0,12647.0,6.111001e+07,13091.0,...,97099.0,5.0,1967.0,88000.0,464000.0,2016.0,376000.0,5672.48,0.055619,2017-01-01
2,14186244,2.0,3.0,1243.0,6059.0,33886168.0,-117823170.0,8432.0,6.059022e+07,21412.0,...,97078.0,6.0,1962.0,85289.0,564778.0,2016.0,479489.0,6488.30,0.005383,2017-01-01
3,12177905,3.0,4.0,2376.0,6037.0,34245180.0,-118240722.0,13038.0,6.037300e+07,396551.0,...,96330.0,0.0,1970.0,108918.0,145143.0,2016.0,36225.0,1777.51,-0.103410,2017-01-01
6,12095076,3.0,4.0,2962.0,6037.0,34145202.0,-118179824.0,63000.0,6.037461e+07,47019.0,...,96293.0,0.0,1950.0,276684.0,773303.0,2016.0,496619.0,9516.26,-0.001011,2017-01-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77568,12412492,2.0,4.0,1633.0,6037.0,33870815.0,-118070858.0,4630.0,6.037555e+07,30267.0,...,96204.0,0.0,1962.0,125466.0,346534.0,2016.0,221068.0,4175.08,0.001082,2017-09-19
77570,11000655,2.0,2.0,1286.0,6037.0,34245368.0,-118282383.0,47405.0,6.037101e+07,12447.0,...,96284.0,0.0,1940.0,70917.0,354621.0,2016.0,283704.0,4478.43,0.020615,2017-09-20
77571,17239384,2.0,4.0,1612.0,6111.0,34300140.0,-118706327.0,12105.0,6.111008e+07,27110.0,...,97116.0,7.0,1964.0,50683.0,67205.0,2016.0,16522.0,1107.48,0.013209,2017-09-21
77572,12773139,1.0,3.0,1032.0,6037.0,34040895.0,-118038169.0,5074.0,6.037434e+07,36502.0,...,96480.0,0.0,1954.0,32797.0,49546.0,2016.0,16749.0,876.43,0.037129,2017-09-21


In [27]:
df.lotsizesquarefeet.fillna(7313, inplace = True)

In [28]:
df.head()

,parcelid,bathroomcnt,bedroomcnt,calculatedfinishedsquarefeet,fips,latitude,longitude,lotsizesquarefeet,rawcensustractandblock,regionidcity,...,regionidzip,roomcnt,yearbuilt,structuretaxvaluedollarcnt,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,logerror,transactiondate
0,14297519,3.5,4.0,3100.0,6059.0,33634931.0,-117869207.0,4506.0,6.059063e+07,53571.0,...,96978.0,0.0,1998.0,485713.0,1023282.0,2016.0,537569.0,11013.72,0.025595,2017-01-01
1,17052889,1.0,2.0,1465.0,6111.0,34449266.0,-119281531.0,12647.0,6.111001e+07,13091.0,...,97099.0,5.0,1967.0,88000.0,464000.0,2016.0,376000.0,5672.48,0.055619,2017-01-01
2,14186244,2.0,3.0,1243.0,6059.0,33886168.0,-117823170.0,8432.0,6.059022e+07,21412.0,...,97078.0,6.0,1962.0,85289.0,564778.0,2016.0,479489.0,6488.30,0.005383,2017-01-01
3,12177905,3.0,4.0,2376.0,6037.0,34245180.0,-118240722.0,13038.0,6.037300e+07,396551.0,...,96330.0,0.0,1970.0,108918.0,145143.0,2016.0,36225.0,1777.51,-0.103410,2017-01-01
6,12095076,3.0,4.0,2962.0,6037.0,34145202.0,-118179824.0,63000.0,6.037461e+07,47019.0,...,96293.0,0.0,1950.0,276684.0,773303.0,2016.0,496619.0,9516.26,-0.001011,2017-01-01


In [29]:
df = df[(df.bedroomcnt > 0) & (df.bathroomcnt > 0) & (df.calculatedfinishedsquarefeet>350)]

In [30]:
df.shape

(52077, 21)

In [55]:
def wrangle_zillow():
    # acquire df
    df = get_zillow_data()
    # only single family
    df = df[df.propertylandusetypeid == 261]
    # at least 1 bed and bath and 350 sqft
    df = df[(df.bedroomcnt > 0) & (df.bathroomcnt > 0) & (df.calculatedfinishedsquarefeet>350)]
    # handle missing values
    df = handle_missing_values(df)
    # drop unnecessary columns
    df = drop_cols(df, ['id','calculatedbathnbr', 'buildingqualitytypeid','finishedsquarefeet12', 'fullbathcnt', 'heatingorsystemtypeid','heatingorsystemdesc','propertycountylandusecode', 'propertylandusetypeid','propertyzoningdesc',  'censustractandblock', 'propertylandusedesc', 'unitcnt'])
    # fill lotsize
    df.lotsizesquarefeet.fillna(7313, inplace = True)
    # properties under 5 million USD
    df = df[df.taxvaluedollarcnt < 5_000_000]
    # add counties
    df['county'] = np.where(df.fips == 6037, 'Los_Angeles',np.where(df.fips == 6059, 'Orange', 'Ventura'))  
    # catch other nulls
    df.dropna(inplace=True)
    # return wrangled df
    return df

In [56]:
df = wrangle_zillow()

/opt/homebrew/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3437: DtypeWarning: Columns (64) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [57]:
df[df.fips != 6037].isnull().sum()

parcelid                        0
bathroomcnt                     0
bedroomcnt                      0
calculatedfinishedsquarefeet    0
fips                            0
latitude                        0
longitude                       0
lotsizesquarefeet               0
rawcensustractandblock          0
regionidcity                    0
regionidcounty                  0
regionidzip                     0
roomcnt                         0
yearbuilt                       0
structuretaxvaluedollarcnt      0
taxvaluedollarcnt               0
assessmentyear                  0
landtaxvaluedollarcnt           0
taxamount                       0
logerror                        0
transactiondate                 0
county                          0
dtype: int64

In [58]:
df.fips

0        6059.0
1        6111.0
2        6059.0
3        6037.0
6        6037.0
          ...  
77568    6037.0
77570    6037.0
77571    6111.0
77572    6037.0
77573    6037.0
Name: fips, Length: 50957, dtype: float64